In [24]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [25]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [26]:
from src.inference import get_feature_store, fetch_predictions
import pandas as pd
from datetime import datetime, timedelta, timezone


def fetch_days_data(days):
    current_date = pd.to_datetime(datetime.now(timezone.utc))
    fetch_data_from = current_date - timedelta(days=(365+days)) 
    fetch_data_to = current_date - timedelta(days=365)
    print(fetch_data_from, fetch_data_to)
    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    # query = query.filter((fg.pickup_hour >= fetch_data_from))
    df = query.read()
    cond = (df["pickup_hour"] >= fetch_data_from) & (df["pickup_hour"] <= fetch_data_to)
    return df[cond]

In [27]:
ts_data = fetch_days_data(180)

2023-09-06 22:54:40.453653+00:00 2024-03-04 22:54:40.453653+00:00
2025-03-04 17:54:40,487 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 17:54:40,565 INFO: Initializing external client
2025-03-04 17:54:40,565 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 17:54:41,223 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (9.61s) 


In [28]:
from src.data_utils import transform_ts_data_info_features_and_target

features, targets = transform_ts_data_info_features_and_target(ts_data, window_size=24*28, step_size=23)

In [29]:
from src.pipeline_utils import get_pipeline
pipeline = get_pipeline()
pipeline.fit(features, targets)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.148500 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 171646
[LightGBM] [Info] Number of data points in the train set: 23920, number of used features: 675
[LightGBM] [Info] Start training from score 16.970234


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x00000217FE499940>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [30]:
from sklearn.metrics import mean_absolute_error
predictions = pipeline.predict(features)

In [31]:
test_mae = mean_absolute_error(targets, predictions)
print(f"{test_mae:.4f}")

2.9240


In [32]:
from src.inference import load_metrics_from_registry 

metric = load_metrics_from_registry()

2025-03-04 17:55:52,995 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 17:55:53,011 INFO: Initializing external client
2025-03-04 17:55:53,011 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 17:55:53,741 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [33]:
metric

{'test_mae': 2.920020213010517}

In [34]:
from src.inference import load_model_from_registry
model = load_model_from_registry()

2025-03-04 17:55:59,193 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 17:55:59,209 INFO: Initializing external client
2025-03-04 17:55:59,211 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 17:55:59,741 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697

In [35]:
import joblib  
import src.config
# Save the pipeline  
joblib.dump(pipeline, config.MODELS_DIR / "lgb_model.pkl")

['C:\\Users\\thano\\Documents\\University\\CDA500\\sp25_taxi-main\\models\\lgb_model.pkl']

In [36]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(features)
output_schema = Schema(targets)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

In [37]:
import hopsworks

project = hopsworks.login(
    project="thanoojl", 
    api_key_value="S62Mv7Bj2L5lyQOF.xsjVCV2ettmgcFDadW6yjhKfLlrvm2egSi72VLxDSxbbLSHWMoiJoVv9EY9ObMrg"
)
model_registry = project.get_model_registry()

2025-03-04 17:56:20,189 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 17:56:20,198 INFO: Initializing external client
2025-03-04 17:56:20,198 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 17:56:20,728 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


In [38]:
model_registry = project.get_model_registry()
model_registry

ModelRegistry(project: 'thanoojl')

In [39]:
modelv2 = model_registry.sklearn.create_model(
    name="taxi_demand_predictor_next_hour",
    metrics={"test_mae": test_mae},
    description="LightGBM regressor V2",
    input_example=features.sample(),
    model_schema=model_schema,
)

In [40]:
modelv2.save('C:\\Users\\thano\\Documents\\University\\CDA500\\sp25_taxi-main\\models\\lgb_model.pkl')

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/315669 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/2049 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/51418 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1214697/models/taxi_demand_predictor_next_hour/3


Model(name: 'taxi_demand_predictor_next_hour', version: 3)

In [41]:
from src.inference import load_model_from_registry
model = load_model_from_registry(-1)

2025-03-04 17:56:54,913 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 17:56:54,919 INFO: Initializing external client
2025-03-04 17:56:54,919 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 17:56:55,555 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697

In [20]:
models = model_registry.get_models(name=config.MODEL_NAME)

In [21]:
models

[Model(name: 'taxi_demand_predictor_next_hour', version: 1),
 Model(name: 'taxi_demand_predictor_next_hour', version: 2)]

In [22]:
max(models, key=lambda model: model.version)

Model(name: 'taxi_demand_predictor_next_hour', version: 2)

In [23]:
load_metrics_from_registry()

2025-03-04 05:09:22,041 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 05:09:22,050 INFO: Initializing external client
2025-03-04 05:09:22,053 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 05:09:22,790 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697


{'test_mae': 2.920020213010517}